<a href="https://colab.research.google.com/github/shobhitsundriyal/rand_ideas/blob/master/Machine_translation_tf(TE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget "http://www.manythings.org/anki/hin-eng.zip"

--2020-05-26 00:38:23--  http://www.manythings.org/anki/hin-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:3033::6818:6dc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126449 (123K) [application/zip]
Saving to: ‘hin-eng.zip’

hin-eng.zip         100%[===================>] 123.49K   230KB/s    in 0.5s    

2020-05-26 00:38:24 (230 KB/s) - ‘hin-eng.zip’ saved [126449/126449]



In [6]:
!unzip "hin-eng.zip"

Archive:  hin-eng.zip
  inflating: hin.txt                 
  inflating: _about.txt              


In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c "tweet-sentiment-extraction"
!unzip 'train.csv.zip'

mkdir: cannot create directory ‘/root/.kaggle’: File exists
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [0]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [0]:
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
def create_dataset(df, sentiment=None, test_data=False):
  if test_data:
    t = df.text
    t = t.apply(lambda w: preprocess_sentence(str(w)))
    return t.tolist()

  else:
    if sentiment != None:
      t = df.text[df['sentiment'] == sentiment]
      st = df.selected_text[df['sentiment'] == sentiment]
      t = t.apply(lambda w: preprocess_sentence(str(w)))
      st = st.apply(lambda w: preprocess_sentence(str(w)))

    else:
      t = df.text
      st = df.selected_text
      t = t.apply(lambda w: preprocess_sentence(str(w)))
      st = st.apply(lambda w: preprocess_sentence(str(w)))

    return t.tolist(), st.tolist()

In [6]:
pos_text, pos_sel_text = create_dataset(train_df, 'positive')
neg_text, neg_sel_text = create_dataset(train_df, 'negative')

pos_text = pos_text + neg_text
pos_sel_text = pos_sel_text + neg_sel_text #trying both 

print(pos_text[-1])
print(pos_sel_text[-1])

<start> i ve wondered about rake to . the client has made it clear . net only , don t force devs to learn a new lang agile ccnet <end>
<start> , don t force <end>


In [7]:
print(len(pos_text))

16363


In [0]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

In [0]:
def load_dataset(path, num_examples=None):#xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
vocab = create_dataset(train_df, test_data=True) + create_dataset(test_df, test_data=True)

In [0]:
lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
                 filters='')
lang_tokenizer.fit_on_texts(vocab)

pos_seq_tensors = lang_tokenizer.texts_to_sequences(pos_text)
pos_sel_seq_tensors = lang_tokenizer.texts_to_sequences(pos_sel_text)

pos_seq_tensors = tf.keras.preprocessing.sequence.pad_sequences(pos_seq_tensors,
                                                         padding='post')
pos_sel_seq_tensors = tf.keras.preprocessing.sequence.pad_sequences(pos_sel_seq_tensors,
                                                         padding='post')

In [12]:
print(pos_seq_tensors.shape)
print(pos_sel_seq_tensors.shape)

(16363, 89)
(16363, 65)


In [13]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(pos_seq_tensors, pos_sel_seq_tensors, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

13090 13090 3273 3273


In [0]:
def convert(vocab, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, vocab.index_word[t]))

In [15]:
print ("Input Language; index to word mapping")
convert(lang_tokenizer, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(lang_tokenizer, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <start>
48 ----> your
344 ----> making
22 ----> me
670 ----> jealous
38 ----> now
4 ----> i
84 ----> want
60 ----> one
5 ----> !
83 ----> how
178 ----> long
25 ----> have
59 ----> u
216 ----> guys
108 ----> been
1432 ----> married
14 ----> ?
3 ----> <end>

Target Language; index to word mapping
2 ----> <start>
670 ----> jealous
3 ----> <end>


In [0]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(lang_tokenizer.word_index)+1
vocab_tar_size = len(lang_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [17]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 89]), TensorShape([64, 65]))

---

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [19]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 89, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [21]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 89, 1)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [23]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 26469)


In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [0]:
# Checkpointing
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)##

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [27]:
EPOCHS = 50

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.7613
Epoch 1 Batch 100 Loss 0.4195
Epoch 1 Batch 200 Loss 0.5154
Epoch 1 Loss 0.4822
Time taken for 1 epoch 259.896466255188 sec

Epoch 2 Batch 0 Loss 0.4204
Epoch 2 Batch 100 Loss 0.3560
Epoch 2 Batch 200 Loss 0.4124
Epoch 2 Loss 0.4322
Time taken for 1 epoch 171.78362607955933 sec

Epoch 3 Batch 0 Loss 0.4739
Epoch 3 Batch 100 Loss 0.4448
Epoch 3 Batch 200 Loss 0.4871
Epoch 3 Loss 0.4102
Time taken for 1 epoch 169.77469444274902 sec

Epoch 4 Batch 0 Loss 0.3930
Epoch 4 Batch 100 Loss 0.3755
Epoch 4 Batch 200 Loss 0.3504
Epoch 4 Loss 0.3843
Time taken for 1 epoch 171.8023796081543 sec

Epoch 5 Batch 0 Loss 0.3036
Epoch 5 Batch 100 Loss 0.3095
Epoch 5 Batch 200 Loss 0.2996
Epoch 5 Loss 0.3558
Time taken for 1 epoch 169.94661235809326 sec

Epoch 6 Batch 0 Loss 0.3375
Epoch 6 Batch 100 Loss 0.3461
Epoch 6 Batch 200 Loss 0.2888
Epoch 6 Loss 0.3233
Time taken for 1 epoch 172.542542219162 sec

Epoch 7 Batch 0 Loss 0.2888
Epoch 7 Batch 100 Loss 0.4064
Epoch 7 Batch 200

In [0]:
max_length_inp = pos_seq_tensors.shape[1]
max_length_targ = pos_sel_seq_tensors.shape[1]
inp_lang = lang_tokenizer
targ_lang = lang_tokenizer

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))
  print('-'*40)

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  #plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [33]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
p_test = test_df[test_df['sentiment'] != 'neutral']

In [35]:
k = 40
for t in p_test.text.values:
  translate(t)
  k -= 1
  if k == 0:
    break

Input: <start> shanghai is also really exciting precisely skyscrapers galore . good tweeps in china sh bj . <end>
Predicted translation: bored <end> 
----------------------------------------
Input: <start> recession hit veronique branquinho , she has to quit her company , such a shame ! <end>
Predicted translation: wrong . <end> 
----------------------------------------
Input: <start> happy bday ! <end>
Predicted translation: happy <end> 
----------------------------------------
Input: <start> http twitpic . com w p i like it ! ! <end>
Predicted translation: i like it ! ! <end> 
----------------------------------------
Input: <start> that s great ! ! weee ! ! visitors ! <end>
Predicted translation: s great ! ! <end> 
----------------------------------------
Input: <start> i think everyone hates me on here lol <end>
Predicted translation: i think your dog me on here lol <end> 
----------------------------------------
Input: <start> soooooo wish i could , but im in school and myspace is 

In [0]:
k = 30
for t in test_df.text.values:
  translate(t)
  k -= 1
  if k == 0:
    break

Input: <start> last session of the day http twitpic . com ezh <end>
Predicted translation: not a day <end> 
----------------------------------------
Input: <start> shanghai is also really exciting precisely skyscrapers galore . good tweeps in china sh bj . <end>
Predicted translation: good <end> 
----------------------------------------
Input: <start> recession hit veronique branquinho , she has to quit her company , such a shame ! <end>
Predicted translation: loved her to have a safe to have a safe to have a safe to have a safe to have a safe to have a safe to have a safe to have a safe to have a safe to 
----------------------------------------
Input: <start> happy bday ! <end>
Predicted translation: happy ! <end> 
----------------------------------------
Input: <start> http twitpic . com w p i like it ! ! <end>
Predicted translation: i like it ! <end> 
----------------------------------------
Input: <start> that s great ! ! weee ! ! visitors ! <end>
Predicted translation: great ! ! 